In [1]:
import itertools
from math import sqrt
import numpy as np
from numpy import arange, pi, sin, cos, arccos

GOLDEN_RATIO = (1 + 5**0.5)/2

def fibonacci_sequence_sphere(center, radius, n_points):
    i = arange(0, n_points)
    theta = 2 *pi * i / GOLDEN_RATIO
    phi = arccos(1 - 2*(i+0.5)/n_points)
    x = radius * sin(phi) * cos(theta) + center
    y = radius * sin(phi) * sin(theta) + center
    z = radius * cos(phi) + center

    lms = np.vstack((x, y, z)).T

    # rounding of matrix values to account for pixel rounding errors
    lms[lms < center - radius + 0.5] = center - radius + 0.5
    lms[lms > center + radius - 0.5] = center + radius - 0.5

    return lms

def uniform_distribution_faces_cube(begin_corner, end_corner, n_points_dim):
    diff = np.array(end_corner) - np.array(begin_corner)
    lms = []

    for corner, mult in zip([begin_corner, end_corner], [1, -1]):
        for corner_displacement in [np.array(x) for x in [(1, 1, 0), (1, 0, 1), (0, 1, 1)]]:
            indices = list(np.nonzero(corner_displacement)[0])
            indices.append(np.where(corner_displacement == 0)[0][0])
            end_corner_ = corner + mult * corner_displacement * diff
            points = np.array([None, None, None])
            points[indices[0]] = np.linspace(corner[indices[0]], end_corner_[indices[0]], n_points_dim)
            points[indices[1]] = np.linspace(corner[indices[1]], end_corner_[indices[1]], n_points_dim)
            points[indices[2]] = [end_corner_[indices[2]]]
            
            lms.extend(list(itertools.product(*points)))

    return np.unique(np.array(lms), axis=0)

@np.vectorize
def cube_sphere_mapping(x, y, z):
    return x*sqrt(1 - y**2/2 - z**2/2 + y**2*z**2/3), y*sqrt(1 - z**2/2 - x**2/2 + z**2*x**2/3), z*sqrt(1 - x**2/2 - y**2/2 + x**2*y**2/3)

def scale_and_translate(points, scale, translation):
    return points * scale + translation

def write_landmarks_to_file(*landmarks, filename):
    total_size = sum([len(l) for l in landmarks])
    with open(filename, 'w') as f:
        f.write('point\n')
        f.write(f'{total_size}\n')
        for landmarks_ in landmarks:
            for lm in landmarks_:
                f.write(f'{lm[0]} {lm[1]} {lm[2]}\n')                

In [2]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [5]:
lms = uniform_distribution_faces_cube((-1, -1, -1), (1, 1, 1), 3)
color = [i for i in range(len(lms))]
fig = px.scatter_3d(x=lms[:,0], y=lms[:,1], z=lms[:,2], color=color)
fig.show()

In [6]:
lms_sphere = np.array(cube_sphere_mapping(*lms.T)).T
fig = px.scatter_3d(x=lms_sphere[:,0], y=lms_sphere[:,1], z=lms_sphere[:,2], color=color)
fig.show()

In [34]:
unit_cube_points = uniform_distribution_faces_cube((-1, -1, -1), (1, 1, 1), 5)
unit_sphere_points = np.array(cube_sphere_mapping(*unit_cube_points.T)).T

# Scene 01

lms_cube = scale_and_translate(unit_cube_points, 10, 15)

lms_sphere_moving = scale_and_translate(unit_sphere_points, 8, 15)
lms_sphere_fixed = scale_and_translate(unit_sphere_points, 5, 15)

write_landmarks_to_file(lms_cube, lms_sphere_moving, filename='01_Moving.txt')
write_landmarks_to_file(lms_cube, lms_sphere_fixed, filename='01_Fixed.txt')

In [7]:
unit_cube_points = uniform_distribution_faces_cube((-1, -1, -1), (1, 1, 1), 3)
unit_sphere_points = np.array(cube_sphere_mapping(*unit_cube_points.T)).T

#  Scene 02

lms_cube_moving = scale_and_translate(unit_cube_points, 10, 15)

lms_cube_fixed = scale_and_translate(unit_cube_points, 6, 15)

lms_sphere_moving = scale_and_translate(unit_sphere_points, 6, 15)

lms_sphere_fixed = scale_and_translate(unit_sphere_points, 13, 15)

write_landmarks_to_file(lms_cube_moving, lms_sphere_moving, filename='02_Moving.txt')
write_landmarks_to_file(lms_sphere_fixed, lms_cube_fixed, filename='02_Fixed.txt')